In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bam,IR,29.1060,58.3570,83.52,14,3,7.00,clear sky
1,1,Busselton,AU,-33.6500,115.3333,53.74,67,42,5.32,scattered clouds
2,2,Kapaa,US,22.0752,-159.3190,82.47,68,40,12.66,scattered clouds
3,3,Bluff,NZ,-46.6000,168.3333,44.80,75,11,20.45,few clouds
4,4,Caravelas,BR,-17.7125,-39.2481,71.31,61,100,7.70,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip?  "))
max_temp=float(input("What is the maximum temperature you would like for your trip?  "))

What is the minimum temperature you would like for your trip?  70
What is the maximum temperature you would like for your trip?  95


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df['Max Temp']>=min_temp)&\
                                    (city_data_df['Max Temp']<=max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bam,IR,29.1060,58.3570,83.52,14,3,7.00,clear sky
2,2,Kapaa,US,22.0752,-159.3190,82.47,68,40,12.66,scattered clouds
4,4,Caravelas,BR,-17.7125,-39.2481,71.31,61,100,7.70,overcast clouds
5,5,Tefe,BR,-3.3542,-64.7114,86.11,70,75,5.75,broken clouds
6,6,Vaini,TO,-21.2000,-175.2000,71.76,78,75,10.36,broken clouds
12,12,Manaquiri,BR,-3.3167,-60.3500,88.29,61,66,1.28,broken clouds
16,16,Castanas,PH,13.8822,121.5517,77.18,89,100,4.70,overcast clouds
25,25,Bonthe,SL,7.5264,-12.5050,77.59,81,85,8.84,overcast clouds
27,27,Itarema,BR,-2.9248,-39.9167,78.40,75,92,18.01,overcast clouds
28,28,Yellowknife,CA,62.4560,-114.3525,72.12,43,20,11.50,few clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                371
City                   371
Country                371
Lat                    371
Lng                    371
Max Temp               371
Humidity               371
Cloudiness             371
Wind Speed             371
Current Description    371
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bam,IR,83.52,clear sky,29.1060,58.3570,
2,Kapaa,US,82.47,scattered clouds,22.0752,-159.3190,
4,Caravelas,BR,71.31,overcast clouds,-17.7125,-39.2481,
5,Tefe,BR,86.11,broken clouds,-3.3542,-64.7114,
6,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,
12,Manaquiri,BR,88.29,broken clouds,-3.3167,-60.3500,
16,Castanas,PH,77.18,overcast clouds,13.8822,121.5517,
25,Bonthe,SL,77.59,overcast clouds,7.5264,-12.5050,
27,Itarema,BR,78.40,overcast clouds,-2.9248,-39.9167,
28,Yellowknife,CA,72.12,few clouds,62.4560,-114.3525,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    # 6c. Get latitude and longitude from DataFrame
    params['location']=f'{lat}, {lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]= hotels['results'][0]['name']
    except(IndexError):
        print("Hotel not found....skipping")
        

Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping
Hotel not found....skipping


In [31]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bam,IR,83.52,clear sky,29.1060,58.3570,Bam Tourist Hotel
2,Kapaa,US,82.47,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Caravelas,BR,71.31,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
5,Tefe,BR,86.11,broken clouds,-3.3542,-64.7114,Aracati Hotel
6,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Manaquiri,BR,88.29,broken clouds,-3.3167,-60.3500,
16,Castanas,PH,77.18,overcast clouds,13.8822,121.5517,The Ponce Farm
25,Bonthe,SL,77.59,overcast clouds,7.5264,-12.5050,Bonthe Holiday Village
27,Itarema,BR,78.40,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
28,Yellowknife,CA,72.12,few clouds,62.4560,-114.3525,The Explorer Hotel


In [43]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace(r'^s*$', float('NaN'), regex = True).dropna()
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bam,IR,83.52,clear sky,29.1060,58.3570,Bam Tourist Hotel
2,Kapaa,US,82.47,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Caravelas,BR,71.31,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
5,Tefe,BR,86.11,broken clouds,-3.3542,-64.7114,Aracati Hotel
6,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
16,Castanas,PH,77.18,overcast clouds,13.8822,121.5517,The Ponce Farm
25,Bonthe,SL,77.59,overcast clouds,7.5264,-12.5050,Bonthe Holiday Village
27,Itarema,BR,78.40,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
28,Yellowknife,CA,72.12,few clouds,62.4560,-114.3525,The Explorer Hotel
29,Carutapera,BR,77.85,broken clouds,-1.1950,-46.0200,Lidera


In [45]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt//><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>and<dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# hotel_info
# locations.head(10)

In [56]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))